In [1]:
import cv2
from matplotlib import pyplot as plt
import numpy as np

: 

In [2]:
import wandb
from controller import Robot

from WebotsEnvironment import WebotsEnvironment
from WebotsGymEnv import WebotsGymEnv

env = WebotsEnvironment()
    
obs = env.reset()[0]
env.reset()

In [ ]:
import math

import cv2
import numpy as np

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


In [ ]:
def perspective_transform(image):
    height, width = image.shape[:2]
    
    src = np.float32([(0, height),
                        ((width * 0.25) + 50, height * 0.50),
                        ((width * 0.75) - 50, height * 0.50),
                        (width, height),])    # top-right
    
    dst = np.float32([(0, height),
                        (100, 100),
                        (width - 100, 100),
                        (width, height)])
    
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    # Apply the perspective transformation
    warped = cv2.warpPerspective(image, M, (width, height))

    return warped, M, M_inv

In [ ]:
import math
import random

import cv2
import numpy as np

def pipeline(image, center_margin = 50):
    """
    An image processing pipeline which will output
    an image with the lane lines annotated.
    """
    
    height = image.shape[0]
    width = image.shape[1]

    region_of_interest_vertices = [
        (0, height),
        (width * 0.25, height * 0.50),
        (width * 0.75, height * 0.50),
        (width, height),
    ]

    cropped_image = region_of_interest(
        image,
        np.array(
            [region_of_interest_vertices],
            np.int32
        ),
    )

    upper_yellow = np.array([215, 220, 237])
    lower_yellow = np.array([80, 80, 208])

    mask = cv2.inRange(cropped_image, lower_yellow, upper_yellow)
    masked = cv2.bitwise_and(image, image , mask=mask)

    gray_image = cv2.cvtColor(masked, cv2.COLOR_RGB2GRAY)

    warped, M, M_inv = perspective_transform(gray_image)
    cannyed_image = cv2.Canny(warped, 850, 250, apertureSize=3)
    
    lines = cv2.HoughLinesP(cannyed_image, rho=2.5, theta=np.pi/180, threshold=15, minLineLength=20, maxLineGap=15)
        
    left_points = []
    right_points = []

    center = cannyed_image.shape[1] // 2

    left_weight = 0
    right_weight = 0

    def calc_weight(x):
        return abs(x - center)
    
    if lines is None:
        return left_weight,right_weight,gray_image,0,cropped_image,cannyed_image
    
    for line in lines:
        for x1, y1, x2, y2 in line:

            if abs(x1 - x2) > 5 and abs(x1 - x2) < 10:
                continue  # Skip vertical lines to avoid division by zero
            
            if x1 < center - center_margin :  # Left line
                left_points.append((x1, y1))
                left_weight += calc_weight(x1)
            if x2 < center - center_margin :  # Left line
                left_points.append((x2, y2))
                left_weight += calc_weight(x2)
            
            if x1 > center + center_margin :  # Left line
                right_points.append((x1, y1))
                right_weight += calc_weight(x1)
                
            if x2 > center + center_margin :  # Left line
                right_points.append((x2, y2))
                right_weight += calc_weight(x2)
                
    if len(right_points) > 0:
        right_weight = right_weight / len(right_points)

    if len(left_points) > 0:
        left_weight = left_weight / len(left_points)
         
    return left_weight,right_weight,gray_image,len(lines),cropped_image,cannyed_image

In [ ]:
def calculate_steering_angle(image_width, left_weight, right_weight):
    final_weight = right_weight - left_weight

    diff = image_width / 2
    angle = final_weight / diff
    return angle

In [ ]:
radian = 0

In [ ]:
def get_heading_vector():
        
    compass = np.array(env.compass.getValues()[0:2], dtype=np.float16)
    # Rotate the vector
    # rotated_vector = [-compass[1] , -compass[0]]

    return compass

In [ ]:
import datetime
import time

env.step(action=[1,radian],max_steps=1000)
camera = env.camera

pointer = camera.getImage()
        
image_array = np.frombuffer(pointer, dtype=np.uint8)

# Reshape the array to the shape (height, width, 4) assuming the image has 4 channels (RGBA)
img = image_array.reshape((camera.height, camera.width, 4))
rgb_array = img[:, :, :3]

cv2_img = rgb_array

left_weight,right_weight,gray_image,lines,cropped_image,cannyed_image = pipeline(cv2_img, center_margin=40)

In [ ]:
calculate_steering_angle(gray_image.shape[1],left_weight,right_weight)

0.0

In [ ]:
previous_position = env.previous_pos
current_position = np.array(env.gps.getValues()[0:2])
compass_heading = get_heading_vector()
print(compass_heading)

# Calculate the displacement vector
displacement = current_position - previous_position

# Normalize the compass heading vector to get the forward direction
forward_direction = compass_heading / np.linalg.norm(compass_heading)

# Extend the forward direction to 3D (XY plane, Z=0)
forward_direction_3d = np.array([forward_direction[0], forward_direction[1]])

# Project the displacement vector onto the forward direction
forward_distance = np.dot(displacement, forward_direction_3d)
forward_distance

[-1.00e+00  2.56e-06]


0.028575993445373444

In [ ]:
left_weight,right_weight,lines

(0, 0, 0)

In [ ]:
height, width = cv2_img.shape[:2]
y_eval = height  # Evaluate at the bottom of the image

if left_fit is not None and right_fit is not None:
    left_x = np.polyval(left_fit, y_eval)
    right_x = np.polyval(right_fit, y_eval)
    lane_center = (left_x + right_x) / 2
elif left_fit is not None:
    lane_center = np.polyval(left_fit, y_eval)
elif right_fit is not None:
    lane_center = np.polyval(right_fit, y_eval)
else:
    lane_center = width / 2

# Calculate angle in radians
angle = np.arctan((lane_center - width / 2) / y_eval)

# Convert angle to degrees
angle_degrees = np.degrees(angle)

In [ ]:
steering_angle = calculate_steering_angle(cv2_img.shape[:2], left_fit if left_r_squared >= right_r_squared else None, 
                                                        right_fit if right_r_squared >= left_r_squared else None)
radian = steering_angle * (np.pi/180)